In [1]:
import pandas as pd 
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import os, sys, time
sys.path.append("..")
from all_funcs import util
from model import Generator, Discriminator, train_discriminator, train_generator
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers
from sklearn.impute import SimpleImputer
tf.keras.backend.set_floatx('float64')
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)

In [2]:
# from numba import cuda
# os.environ['CUDA_VISIBLE_DEVICES']="0"
# print(tf.test.is_gpu_available())

True


In [3]:
df=pd.read_csv("../dataset/df_noOutliner_ana.csv",index_col=0)
df, imp_mode, imp_mean=util.FeatureArrange(df)

In [4]:
## reduce redundant features which can be assembled
dataset=df.drop(['NIHTotal','THD_ID','cortical_CT', 'subcortical_CT',
              'circulation_CT', 'CT_find', 'watershed_CT', 'Hemorrhagic_infarct_CT',
              'CT_left', 'CT_right',],axis=1)

In [5]:
sc = MinMaxScaler()
dataset.loc[:,dataset.columns!='elapsed_class'] = sc.fit_transform(dataset.loc[:,dataset.columns!='elapsed_class'])


In [6]:
## setting hyperparameter
latent_dim = dataset.shape[1]
epochs = 1000
batch_size= 128
buffer_size = 6000
# save_interval = 50
n_critic = 5
checkpoint_dir = './training_checkpoints'


In [7]:
generator = Generator()
discriminator = Discriminator()

In [8]:
## create Cross Entropy
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [9]:
gen_opt = tf.keras.optimizers.Adam(0.0001,)
disc_opt = tf.keras.optimizers.Adam(0.0001,)

In [10]:
# save checkpoints
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=gen_opt,
                                 discriminator_optimizer=disc_opt,
                                 generator=generator,
                                 discriminator=discriminator)

In [11]:
# separate to 0,1 dataset
data_1=dataset.loc[dataset['elapsed_class']==1]
data_0=dataset.loc[dataset['elapsed_class']==0]

In [12]:
for epoch in range(epochs):
    start = time.time()
    disc_loss = 0
    gen_loss = 0

    # resample the dataset
    data1_shape_0 = data_1.sample(data_0.shape[0])
    df_same_shape = pd.concat([data1_shape_0, data_0])

    # slices to data and labels
    df_same_ = df_same_shape.iloc[:, :-1].to_numpy()
    org_label = df_same_shape.iloc[:, -1].to_numpy().reshape(-1, 1)

    # create batch dataset
    training_dataset = tf.data.Dataset.from_tensor_slices((df_same_, org_label))\
        .shuffle(buffer_size).batch(batch_size, drop_remainder=True)

    for data, labels in training_dataset:

        disc_loss += train_discriminator(data, generator,
                                         discriminator, disc_opt, latent_dim)

        if disc_opt.iterations.numpy() % n_critic == 0:
            gen_loss += train_generator(data, generator,
                                        discriminator, gen_opt, batch_size, latent_dim)
    print('Time for epoch {} is {} sec - gen_loss = {}, disc_loss = {}'.format(epoch + 1, time.time() - start,
                                                                               gen_loss / batch_size,
                                                                               disc_loss / (batch_size*n_critic)))


If you intended to run this layer in float64, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float32 by default, call `tf.keras.backend.set_floatx('float32')`. To change just this layer, pass dtype='float32' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Time for epoch 1 is 0.028001785278320312 sec - gen_loss = -0.0005304864617340972, disc_loss = 1.906838777389753
Time for epoch 2 is 0.008000612258911133 sec - gen_loss = 0.005719191952587004, disc_loss = -0.0023444711882774343
Time for epoch 3 is 0.012000799179077148 sec - gen_loss = 0.010827660591138665, disc_loss = -0.007956516078838844
Time for epoch 4 is 0.008000373840332031 sec - gen_loss = 0.013375352519486768, disc_loss = -0.01033248545507485
Time for epoch 5 is 0.008000373840332031 sec - gen_los

Time for epoch 72 is 0.008000612258911133 sec - gen_loss = 0.0037298130613956453, disc_loss = -0.0007186979805643972
Time for epoch 73 is 0.008000373840332031 sec - gen_loss = 0.003869132226333775, disc_loss = -0.00037842333847293476
Time for epoch 74 is 0.008000612258911133 sec - gen_loss = 0.0036692358746216337, disc_loss = -0.0005717137378517293
Time for epoch 75 is 0.008000373840332031 sec - gen_loss = 0.002893725145085854, disc_loss = -0.0007129799608964441
Time for epoch 76 is 0.008000612258911133 sec - gen_loss = 0.0019421731573222687, disc_loss = -0.0007806218935387041
Time for epoch 77 is 0.008000612258911133 sec - gen_loss = 0.0007994846902531782, disc_loss = -0.0007999076584368316
Time for epoch 78 is 0.008000373840332031 sec - gen_loss = -0.00020610470229926868, disc_loss = -0.0007192843115693401
Time for epoch 79 is 0.008000612258911133 sec - gen_loss = -0.0008766782287439162, disc_loss = -0.001263374533012099
Time for epoch 80 is 0.008000373840332031 sec - gen_loss = -0.0

Time for epoch 143 is 0.008000373840332031 sec - gen_loss = 0.0016327258964702357, disc_loss = -0.00033806639731962434
Time for epoch 144 is 0.008000612258911133 sec - gen_loss = 0.001557169822335928, disc_loss = 0.0004898904913135528
Time for epoch 145 is 0.012000799179077148 sec - gen_loss = 0.001768413297928592, disc_loss = 0.000115937839683223
Time for epoch 146 is 0.008000373840332031 sec - gen_loss = 0.0007883745541568603, disc_loss = -0.0006144894085841542
Time for epoch 147 is 0.008000373840332031 sec - gen_loss = 0.0010271216402905877, disc_loss = -0.0009371111676935967
Time for epoch 148 is 0.008000612258911133 sec - gen_loss = 0.0014826539420555336, disc_loss = -8.47003472332699e-05
Time for epoch 149 is 0.012000560760498047 sec - gen_loss = 0.001390216799015843, disc_loss = 0.0016317025321696574
Time for epoch 150 is 0.004000425338745117 sec - gen_loss = 0.0009723107750331277, disc_loss = -6.896659054467977e-05
Time for epoch 151 is 0.008000373840332031 sec - gen_loss = 0.0

Time for epoch 214 is 0.008000373840332031 sec - gen_loss = 0.008306026817734303, disc_loss = 0.0004513681059838668
Time for epoch 215 is 0.008000612258911133 sec - gen_loss = 0.007693859408429407, disc_loss = 0.0011146824589970026
Time for epoch 216 is 0.008000373840332031 sec - gen_loss = 0.008925778838037433, disc_loss = 0.0002689182354464729
Time for epoch 217 is 0.008000612258911133 sec - gen_loss = 0.008293275304320114, disc_loss = -0.0011841376975469272
Time for epoch 218 is 0.008000612258911133 sec - gen_loss = 0.008043757224484676, disc_loss = -0.0012411258875327852
Time for epoch 219 is 0.008000373840332031 sec - gen_loss = 0.007893878167269533, disc_loss = -0.0011945594033636062
Time for epoch 220 is 0.008000612258911133 sec - gen_loss = 0.007395247693958633, disc_loss = -0.0008951071129306211
Time for epoch 221 is 0.004000186920166016 sec - gen_loss = 0.006472189213687725, disc_loss = 0.001150142060957481
Time for epoch 222 is 0.008000612258911133 sec - gen_loss = 0.0070719

Time for epoch 289 is 0.008000612258911133 sec - gen_loss = 0.005445951321588974, disc_loss = -0.0013793510970244944
Time for epoch 290 is 0.008000373840332031 sec - gen_loss = 0.005082201816347206, disc_loss = -4.312769728785288e-05
Time for epoch 291 is 0.008000373840332031 sec - gen_loss = 0.005203226631218593, disc_loss = -0.0009493204545658092
Time for epoch 292 is 0.008000373840332031 sec - gen_loss = 0.005317509411694929, disc_loss = -0.0008181805217085187
Time for epoch 293 is 0.008000612258911133 sec - gen_loss = 0.005251176899981815, disc_loss = -0.0012959499664158215
Time for epoch 294 is 0.008000612258911133 sec - gen_loss = 0.005638107625614978, disc_loss = -0.0014433426282066098
Time for epoch 295 is 0.016000986099243164 sec - gen_loss = 0.005441212760138381, disc_loss = -0.0009478565246826259
Time for epoch 296 is 0.008000373840332031 sec - gen_loss = 0.005995443706338232, disc_loss = -0.0016988450237170306
Time for epoch 297 is 0.008000612258911133 sec - gen_loss = 0.00

...

Time for epoch 660 is 0.008000612258911133 sec - gen_loss = 0.005716800200347644, disc_loss = -0.0010687001237166403
Time for epoch 661 is 0.008000373840332031 sec - gen_loss = 0.0060053014693694454, disc_loss = -0.001793930501649375
Time for epoch 662 is 0.008000612258911133 sec - gen_loss = 0.006493606686283363, disc_loss = -0.0017774675893777886
Time for epoch 663 is 0.008000373840332031 sec - gen_loss = 0.006039533215087128, disc_loss = -0.0013041190971680027
Time for epoch 664 is 0.008000612258911133 sec - gen_loss = 0.006002709622592079, disc_loss = -0.0016888396687442602
Time for epoch 665 is 0.008000612258911133 sec - gen_loss = 0.006491079784766177, disc_loss = -0.0017869118327942114
Time for epoch 666 is 0.00800013542175293 sec - gen_loss = 0.006150225339209167, disc_loss = -0.0018629930673171185
Time for epoch 667 is 0.008000612258911133 sec - gen_loss = 0.005106016820148791, disc_loss = -0.001551829691061759
Time for epoch 668 is 0.008000373840332031 sec - gen_loss = 0.0052

Time for epoch 733 is 0.008000612258911133 sec - gen_loss = 0.0021798714001099624, disc_loss = -0.0018406762483927509
Time for epoch 734 is 0.008000612258911133 sec - gen_loss = 0.0021573855363023527, disc_loss = -0.0017433896938509968
Time for epoch 735 is 0.008000373840332031 sec - gen_loss = 0.0027335455048836106, disc_loss = -0.0016399076106928957
Time for epoch 736 is 0.008000612258911133 sec - gen_loss = 0.0029489962373926734, disc_loss = -0.0020881559681171233
Time for epoch 737 is 0.008000373840332031 sec - gen_loss = 0.002930958963282745, disc_loss = -0.0020819006482214533
Time for epoch 738 is 0.008000612258911133 sec - gen_loss = 0.003782718422440787, disc_loss = -0.0020631499322465038
Time for epoch 739 is 0.008000612258911133 sec - gen_loss = 0.003690455686860609, disc_loss = -0.0020350727493939477
Time for epoch 740 is 0.008000373840332031 sec - gen_loss = 0.004219122113533399, disc_loss = -0.0015036730511952853
Time for epoch 741 is 0.008000612258911133 sec - gen_loss = 

Time for epoch 810 is 0.008000373840332031 sec - gen_loss = 0.002926723984256057, disc_loss = -0.0014141216920575466
Time for epoch 811 is 0.004000186920166016 sec - gen_loss = 0.0024008493651496592, disc_loss = -0.0008147827161550762
Time for epoch 812 is 0.008000612258911133 sec - gen_loss = 0.002906218347223208, disc_loss = -0.0008756233072085161
Time for epoch 813 is 0.008000612258911133 sec - gen_loss = 0.0026441447616558514, disc_loss = -0.0017134117760316853
Time for epoch 814 is 0.00800013542175293 sec - gen_loss = 0.0030073352007732477, disc_loss = -0.0018349763003233238
Time for epoch 815 is 0.012000799179077148 sec - gen_loss = 0.0025189827985807765, disc_loss = -0.0020493935168473
Time for epoch 816 is 0.008000612258911133 sec - gen_loss = 0.0019767427187088885, disc_loss = -0.0017191640754792696
Time for epoch 817 is 0.008000373840332031 sec - gen_loss = 0.0015638875014863925, disc_loss = -0.0016413428141606
Time for epoch 818 is 0.012000799179077148 sec - gen_loss = 0.001

Time for epoch 884 is 0.008000373840332031 sec - gen_loss = 0.0008245045061431506, disc_loss = -0.001960286156421956
Time for epoch 885 is 0.008000612258911133 sec - gen_loss = 0.0012356757677350282, disc_loss = -0.001686529503049623
Time for epoch 886 is 0.008000612258911133 sec - gen_loss = 0.0013048412938448525, disc_loss = -0.0015577127111186483
Time for epoch 887 is 0.008000373840332031 sec - gen_loss = 0.0020194084986102315, disc_loss = -0.0015994568740775779
Time for epoch 888 is 0.008000612258911133 sec - gen_loss = 0.0021087904358949375, disc_loss = -0.001766377504029038
Time for epoch 889 is 0.008000373840332031 sec - gen_loss = 0.002384060360314414, disc_loss = -0.0018915526163396598
Time for epoch 890 is 0.008000612258911133 sec - gen_loss = 0.0027391190434199494, disc_loss = -0.0018455277787422432
Time for epoch 891 is 0.012000799179077148 sec - gen_loss = 0.0028580040502786774, disc_loss = -0.001701240028935544
Time for epoch 892 is 0.004000186920166016 sec - gen_loss = 0

Time for epoch 958 is 0.004000425338745117 sec - gen_loss = 0.006039329635001457, disc_loss = -0.001804653351009211
Time for epoch 959 is 0.008000373840332031 sec - gen_loss = 0.006699032370060119, disc_loss = -0.001854351859686717
Time for epoch 960 is 0.008000612258911133 sec - gen_loss = 0.006461913607520598, disc_loss = -0.0018627821975532634
Time for epoch 961 is 0.00800013542175293 sec - gen_loss = 0.006155118149770142, disc_loss = -0.0015803540205214257
Time for epoch 962 is 0.008000612258911133 sec - gen_loss = 0.007271090458542068, disc_loss = -0.0017621408459236403
Time for epoch 963 is 0.008000373840332031 sec - gen_loss = 0.007203203565762024, disc_loss = -0.001958997618311076
Time for epoch 964 is 0.008000612258911133 sec - gen_loss = 0.007267105089768604, disc_loss = -0.0013905436827843446
Time for epoch 965 is 0.008000612258911133 sec - gen_loss = 0.0077649488623848404, disc_loss = -0.0017938924706709717
Time for epoch 966 is 0.008000373840332031 sec - gen_loss = 0.00742

In [13]:
arr=np.array([])
noise = tf.random.normal([128, latent_dim])
count=0
for data, labels in training_dataset:
    gen_=generator(noise, data).numpy()
    arr=np.append(arr,gen_,)
    if count==2:
        break
    count+=1

In [14]:
gen_=arr.reshape(-1,62)

In [15]:
output_dataset=pd.DataFrame(np.round(sc.inverse_transform(gen_)),columns=[
'BT_NM', 'HR_NM', 'RR_NM', 'HB_NM', 'HCT_NM', 'PLATELET_NM', 'WBC_NM',
       'PTT1_NM', 'PTT2_NM', 'PTINR_NM', 'ER_NM', 'BUN_NM', 'CRE_NM', 'BMI',
       'age', 'PPD', 'THDA_FL', 'THDH_FL', 'THDI_FL', 'THDAM_FL', 'THDV_FL',
       'THDE_FL', 'THDM_FL', 'THDR_FL', 'THDP_FL', 'THDOO_FL', 'Gender',
       'cortical_ACA_ctr', 'cortical_MCA_ctr', 'subcortical_ACA_ctr',
       'subcortical_MCA_ctr', 'PCA_cortex_ctr', 'thalamus_ctr',
       'brainstem_ctr', 'cerebellum_ctr', 'Watershed_ctr',
       'Hemorrhagic_infarct_ctr', 'cortical_ACA_ctl', 'cortical_MCA_ctl',
       'subcortical_ACA_ctl', 'subcortical_MCA_ctl', 'PCA_cortex_ctl',
       'thalamus_ctl', 'brainstem_ctl', 'cerebellum_ctl', 'Watershed_ctl',
       'Hemorrhagic_infarct_ctl', 'NIHS_1a_in', 'NIHS_1b_in', 'NIHS_1c_in',
       'NIHS_2_in', 'NIHS_3_in', 'NIHS_4_in', 'NIHS_5aL_in', 'NIHS_5bR_in',
       'NIHS_6aL_in', 'NIHS_6bR_in', 'NIHS_7_in', 'NIHS_8_in', 'NIHS_9_in',
       'NIHS_10_in', 'NIHS_11_in',
])
output_dataset

,BT_NM,HR_NM,RR_NM,HB_NM,HCT_NM,PLATELET_NM,WBC_NM,PTT1_NM,PTT2_NM,PTINR_NM,ER_NM,BUN_NM,CRE_NM,BMI,age,PPD,THDA_FL,THDH_FL,THDI_FL,THDAM_FL,THDV_FL,THDE_FL,THDM_FL,THDR_FL,THDP_FL,THDOO_FL,Gender,cortical_ACA_ctr,cortical_MCA_ctr,subcortical_ACA_ctr,subcortical_MCA_ctr,PCA_cortex_ctr,thalamus_ctr,brainstem_ctr,cerebellum_ctr,Watershed_ctr,Hemorrhagic_infarct_ctr,cortical_ACA_ctl,cortical_MCA_ctl,subcortical_ACA_ctl,subcortical_MCA_ctl,PCA_cortex_ctl,thalamus_ctl,brainstem_ctl,cerebellum_ctl,Watershed_ctl,Hemorrhagic_infarct_ctl,NIHS_1a_in,NIHS_1b_in,NIHS_1c_in,NIHS_2_in,NIHS_3_in,NIHS_4_in,NIHS_5aL_in,NIHS_5bR_in,NIHS_6aL_in,NIHS_6bR_in,NIHS_7_in,NIHS_8_in,NIHS_9_in,NIHS_10_in,NIHS_11_in
0,37.0,73.0,22.0,13.0,41.0,192.0,5.0,30.0,27.0,1.0,119.0,25.0,1.0,34.0,72.0,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0
1,37.0,73.0,21.0,15.0,36.0,218.0,8.0,27.0,27.0,1.0,154.0,20.0,1.0,30.0,58.0,70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
2,37.0,74.0,20.0,13.0,45.0,181.0,4.0,27.0,26.0,1.0,117.0,14.0,1.0,34.0,82.0,88.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,37.0,84.0,20.0,11.0,42.0,234.0,14.0,31.0,24.0,1.0,206.0,19.0,1.0,21.0,64.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,4.0,3.0,4.0,0.0,2.0,3.0,2.0,1.0
4,36.0,94.0,21.0,15.0,43.0,145.0,12.0,29.0,27.0,1.0,155.0,20.0,2.0,19.0,75.0,59.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,1.0,4.0,0.0,1.0,3.0,1.0,1.0
5,36.0,77.0,18.0,16.0,42.0,213.0,9.0,34.0,27.0,1.0,200.0,20.0,1.0,27.0,52.0,75.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0
6,36.0,76.0,20.0,14.0,45.0,276.0,9.0,30.0,24.0,1.0,145.0,15.0,1.0,23.0,38.0,47.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
7,37.0,77.0,16.0,14.0,42.0,252.0,9.0,29.0,23.0,1.0,111.0,16.0,1.0,20.0,65.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
8,37.0,99.0,17.0,11.0,44.0,390.0,6.0,32.0,26.0,1.0,121.0,22.0,1.0,22.0,64.0,98.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,37.0,89.0,21.0,16.0,44.0,164.0,12.0,30.0,38.0,1.0,320.0,11.0,2.0,23.0,91.0,60.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,0.0,1.0,3.0,1.0,1.0


In [16]:
# output_dataset.to_csv("../dataset/output_dataset/cWGAN_1d_2_models.csv",encoding='utf_8_sig')